In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/home/thayer/Desktop/DavidCS/ubuntu_partition/code/pydar/')
import pydar

%matplotlib inline
%load_ext autoreload
%autoreload 2

pdal not imported, functions relying on it will fail
opencv was not imported, functions relying on it will fail


In [ ]:
# suffixes
pts_suffix = 'egusnowpit'
trans_suffix = 'slfsnow'

# Project path
project_path = '/media/thayer/Data/mosaic_lidar/ROV/'

# Project name
project_name = 'mosaic_rov_290420.RiSCAN'

# Load the cdf
distro = np.load(os.path.join(os.getcwd(), 'cdf_all.npz'))

